<a href="https://colab.research.google.com/github/alirezasakhaei/SPML_Course2023_Homeworks/blob/main/HW06/JbDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SPML**

**HW6**

**Name: ** Alireza Sakhaeirad

**Student No.: ** 98101714

# JDBA

1. Import libraries

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms

2. Define model

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5), # 16*24*24
            nn.ReLU(),
            nn.Conv2d(16,32,5), # 32*20*20
            nn.ReLU(),
            nn.MaxPool2d(2,2), # 32*10*10
            nn.Conv2d(32,64,5), # 64*6*6
            nn.ReLU(),
            nn.MaxPool2d(2,2) #64*3*3
        )

        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3,100),
            nn.ReLU(),
            nn.Linear(100,10)
        )

    def forward(self,x):
        out = self.layer(x)
        out = out.view(-1,64*3*3)
        out = self.fc_layer(out)

        return out


3. Load original dataset

In [3]:
# todo: Define Batch size & Load MNIST dataset #
from torchvision import datasets, transforms

batch_size = 100
transform = transforms.Compose([transforms.ToTensor()])

# Load the training set
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
mnist_train = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

# Load the test set
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
mnist_test = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)


###########

device = torch.device(torch.cuda.current_device()) if torch.cuda.is_available() else torch.device('cpu')
np.random.seed(0)




100%|██████████| 9912422/9912422 [00:00<00:00, 89655962.57it/s]


Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 115808502.70it/s]

Extracting /root/.pytorch/MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 28467135.90it/s]


Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 20374897.08it/s]


Extracting /root/.pytorch/MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/MNIST_data/MNIST/raw



## Training the substitute model

In [6]:
oracle = CNN().cuda()

# todo: load oracle's checkpoint
weights = torch.load('checkpoint.pth')
oracle.load_state_dict(weights)

<All keys matched successfully>

Defining the model

In [7]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        ### TO DO ###
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(in_features=64 * 5 * 5, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=10)
        #############

    def forward(self, x):
        ### TO DO ###
        x = nn.functional.relu(self.conv1(x))
        x = self.pool1(x)

        x = nn.functional.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 64 * 5 * 5)
        x = nn.functional.relu(self.fc1(x))

        x = self.fc2(x)
        #############
        return x

Implement Jacobian-based Data Augmentation Algorithm

In [8]:
import torch
from torch.autograd import Variable

from torch.utils.data import TensorDataset, DataLoader

In [9]:
def train_JDA(net, X, oracle, epochs=45, lmbd=0.1, max_rho=6):
    ### TO DO ###
    ### Note that you can only use the prediction labels in the training
    ### You can get the prediction labels using net(X).max(1).indices
    def reset_param(net):
        for layer in net.children():
            try:
                layer.reset_parameters()
            except:
                pass

    def train_one_epoch(net, loader, opt, criterian):
        # global y
        # global X
        reset_param(net)
        loader = make_the_loader(X)


        for epoch in range(epochs):
            acc = true_acc(net)
            fidelity = compute_accuracy(net, X, oracle)
            print(f'\t epoch {epoch}: True acc: {round(acc, 3)} / Fidelity: {fidelity}', end='\n')

            net.train()
            for data in loader:
                data = data.to(device)
                # la = la.to(device)
                _, label = oracle(data).max(1)

                output = net(data)
                loss = criterian(output, label)
                net.zero_grad()
                loss.backward()
                opt.step()



    def make_the_loader(dataset):
        if not isinstance(dataset, torch.Tensor):
            dataset = torch.tensor(dataset)
        # dataset = TensorDataset(x)
        dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
        return dataloader

    def dataset_augmentation(net, X, oracle):
        # global y, X
        loader = make_the_loader(X)
        for data in loader:
            data = data.to(device)
            data.requiers_grad = False
            _, labels = oracle(data).max(1)

            data = Variable(data.detach(),requires_grad = True)
            pred = net(data)

            pred_selected = torch.gather(pred, dim=1, index=labels.view(-1, 1)).squeeze()
            quasi_loss = torch.sum(pred_selected)

            net.zero_grad()
            quasi_loss.backward()
            new_data = data - lmbd * data.grad.sign()
            X = torch.cat((X, new_data), dim=0)

    X = X.to(device)
    y = torch.tensor([]).to(device)
    loader = make_the_loader(X)
    for data in loader:
        data = data.to(device)
        _, labels = oracle(data).max(1)
        y = torch.cat((y, labels))

    opt = torch.optim.Adam(net.parameters(), lr=0.001)
    criterian = torch.nn.CrossEntropyLoss()
    for i in range(max_rho):
        print(F'Round {i}:', end='\n')
        train_one_epoch(net, X, opt, criterian)
        dataset_augmentation(net, X, oracle)
    #############

# This function is added too
def true_acc(model):
    model.eval()

    c = 0
    t = 0
    for x, y in mnist_test:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        _, pred = pred.max(1)
        num = (pred == y).sum().item()
        c += num
        t += pred.shape[0]

    return c/t

def compute_accuracy(net, X, oracle):
    ### TO DO ###
    net.eval()
    if not isinstance(X, torch.Tensor):
        X = torch.tensor(X).to(device)
    X = X.to(device)

    correct = 0
    total = 0
    for i, x in enumerate(X):
        _, label = oracle(x.unsqueeze(dim=0).to(device)).max(1)
        _, pred = net(x.unsqueeze(dim=0).to(device)).max(1)
        total += 1
        if pred == label:
            correct += 1

    return correct / total
    #############

In [10]:
### TO DO ###
### Select 100 images with their labels from the dataset, randomly
JDA_train_X, JDA_train_y = next(iter(mnist_test))
#############

net = Net().to(device)

In [11]:
train_JDA(net, JDA_train_X, oracle)

Round 0:
	 epoch 0: True acc: 0.136 / Fidelity: 0.01
	 epoch 1: True acc: 0.133 / Fidelity: 0.5
	 epoch 2: True acc: 0.315 / Fidelity: 0.73
	 epoch 3: True acc: 0.375 / Fidelity: 0.8
	 epoch 4: True acc: 0.415 / Fidelity: 0.9
	 epoch 5: True acc: 0.396 / Fidelity: 0.91
	 epoch 6: True acc: 0.405 / Fidelity: 0.93
	 epoch 7: True acc: 0.417 / Fidelity: 0.9
	 epoch 8: True acc: 0.386 / Fidelity: 0.89
	 epoch 9: True acc: 0.383 / Fidelity: 0.99
	 epoch 10: True acc: 0.407 / Fidelity: 1.0
	 epoch 11: True acc: 0.405 / Fidelity: 1.0
	 epoch 12: True acc: 0.412 / Fidelity: 1.0
	 epoch 13: True acc: 0.41 / Fidelity: 1.0
	 epoch 14: True acc: 0.411 / Fidelity: 1.0
	 epoch 15: True acc: 0.397 / Fidelity: 1.0
	 epoch 16: True acc: 0.427 / Fidelity: 1.0
	 epoch 17: True acc: 0.41 / Fidelity: 1.0
	 epoch 18: True acc: 0.413 / Fidelity: 1.0
	 epoch 19: True acc: 0.411 / Fidelity: 1.0
	 epoch 20: True acc: 0.414 / Fidelity: 1.0
	 epoch 21: True acc: 0.414 / Fidelity: 1.0
	 epoch 22: True acc: 0.413 /

In [12]:
### TO DO ###
### load the whole test and train dataset in one full batch

complete_train = torch.utils.data.DataLoader(trainset, batch_size = len(trainset), shuffle=True)
train_X, train_y = next(iter(complete_train))

complete_test = torch.utils.data.DataLoader(testset, batch_size = len(testset), shuffle=True)
test_X, test_y = next(iter(complete_test))
#############

In [13]:
### TO DO ###
### compute accuracy of net on train and test
def true_acc_calc(model, loader):
    model.eval()

    c = 0
    t = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        _, pred = pred.max(1)
        num = (pred == y).sum().item()
        c += num
        t += pred.shape[0]

    return c/t


train_acc = true_acc_calc(net, complete_train)
test_acc = true_acc_calc(net, complete_test)

print(f'True train accuracy is {round(train_acc, 2)}, and true test accuracy is {round(test_acc, 2)}')
#############

True train accuracy is 0.41, and true test accuracy is 0.42
